Mercedes: logs, monitorizar accesos no autorizados

# Normalización de tablas en forma normal de Boyce Codd

### Importación de Librerías y Configuración Inicial

In [32]:
import pandas as pd
import numpy as np
import warnings
import os
import re
from pathlib import Path
import json
from datetime import datetime

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

### Lectura de los datos limpios y con ingeniería de características

In [33]:
# Carga de datos
file_path = '../cleaned_data/salud_mental_featured.csv'
df = pd.read_csv(file_path)

In [34]:
df.head()

,comunidad_autonoma,fecha_nacimiento,sexo,fecha_ingreso,circunstancia_contacto,fecha_fin_contacto,tipo_alta,estancia_dias,diagnostico_principal,categoria,diagnostico_2,diagnostico_3,diagnostico_4,diagnostico_5,diagnostico_6,diagnostico_7,diagnostico_8,diagnostico_9,diagnostico_10,diagnostico_11,diagnostico_12,diagnostico_13,diagnostico_14,fecha_intervencion,procedimiento_1,procedimiento_2,procedimiento_3,procedimiento_4,procedimiento_5,procedimiento_6,procedimiento_7,procedimiento_8,procedimiento_9,procedimiento_10,procedimiento_11,procedimiento_12,procedimiento_13,procedimiento_14,procedimiento_15,procedimiento_16,procedimiento_17,procedimiento_18,procedimiento_19,procedimiento_20,grd_apr,cdm_apr,nivel_severidad_apr,riesgo_mortalidad_apr,servicio,edad,coste_apr,cie,numero_registro_anual,centro_recodificado,cip_sns_recodificado,pais_nacimiento,pais_residencia,fecha_inicio_contacto,regimen_financiacion,procedencia,continuidad_asistencial,ingreso_en_uci,dias_uci,diagnostico_15,diagnostico_16,diagnostico_17,diagnostico_18,diagnostico_19,diagnostico_20,poa_diagnostico_principal,poa_diagnostico_2,poa_diagnostico_3,poa_diagnostico_4,poa_diagnostico_5,poa_diagnostico_6,poa_diagnostico_7,poa_diagnostico_8,poa_diagnostico_9,poa_diagnostico_10,poa_diagnostico_11,poa_diagnostico_12,poa_diagnostico_13,poa_diagnostico_14,poa_diagnostico_15,poa_diagnostico_16,poa_diagnostico_17,poa_diagnostico_18,poa_diagnostico_19,poa_diagnostico_20,procedimiento_externo_1,procedimiento_externo_2,procedimiento_externo_3,tipo_grd_apr,peso_espanol_apr,edad_en_ingreso,mes_ingreso,tipo_alta_desc,menor_edad,nacido_espana,larga_estancia,cat_esquizofrenia,cat_trastorno_neurotico,cat_trastorno_personalidad,cat_trastorno_mental,cat_trastorno_humor,cat_trastorno_emocional,num_diagnosticos_secundarios,num_procedimientos,complejidad_general,categoria_complejidad,grupo_etario
0,andalucia,1951-08-17,mujer,2016-01-01,1,2016-01-08,1,7,F25.0,"Esquizofrenia, trastornos esquizotípicos y tra...",Z63.79,Z91.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,2,1,PSQ,74,6340,10,8537155.0,-2088791444897189888,109457269-593755146,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,E,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.393611,64,1,domicilio,0,1,1,1,0,0,0,0,0,2,0,0.85,baja,adulto mayor
1,andalucia,1929-03-20,mujer,2016-01-01,1,2016-01-08,1,7,F41.9,"Trastornos neuróticos, trastornos relacionados...",I11.9,I35.8,E11.9,I87.2,Z95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4B02XSZ,B246ZZZ,4A02X4Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,756,19,1,2,CAR,96,2771,10,8992115.0,-1166333372325380096,-1589750168781380096,desconocido,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.609264,86,1,domicilio,0,0,1,0,1,0,0,0,0,5,3,2.35,media,geriatrico
2,andalucia,1976-11-25,varon,2016-01-01,1,2016-01-11,1,10,F60.2,Trastornos de la personalidad y del comportami...,F19.288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,752,19,2,1,PSQ,48,4009,10,8998349.0,17490445801063320188,-5406560181117020160,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.881297,39,1,domicilio,0,1,1,0,0,1,0,0,0,1,0,0.55,baja,adulto medio
3,andalucia,1976-11-10,mujer,2016-01-01,1,2016-01-27,1,26,F20.0,"Esquizofrenia, trastornos esquizotípicos y tra...",C07,F17.210,F12.20,F14.10,F10.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,1,2,PSQ,48,6073,10,8800205.0,-3960068041784730112,-1823171082,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,S,NaN,NaN,NaN,NaN,NaN,

In [35]:
df.describe()

,circunstancia_contacto,tipo_alta,estancia_dias,grd_apr,cdm_apr,nivel_severidad_apr,riesgo_mortalidad_apr,edad,coste_apr,cie,numero_registro_anual,regimen_financiacion,procedencia,continuidad_asistencial,ingreso_en_uci,dias_uci,peso_espanol_apr,edad_en_ingreso,mes_ingreso,menor_edad,nacido_espana,larga_estancia,cat_esquizofrenia,cat_trastorno_neurotico,cat_trastorno_personalidad,cat_trastorno_mental,cat_trastorno_humor,cat_trastorno_emocional,num_diagnosticos_secundarios,num_procedimientos,complejidad_general
count,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.0,2.121000e+04,21210.000000,21210.000000,21210.000000,21210.000000,100.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000,21210.000000
mean,1.104715,1.262706,15.459500,751.342999,18.991372,1.535974,1.057001,51.919142,5453.106554,10.0,9.263254e+06,1.070297,22.031730,8.409147,2.119708,3.510000,1.196840,43.675625,6.282933,0.003300,0.755493,0.697124,0.430269,0.098161,0.153135,0.035078,0.246299,0.030269,3.297831,0.442763,1.226146
std,0.306193,1.074573,19.881648,33.577787,0.954612,0.583499,0.275185,14.135110,1561.748400,0.0,4.702400e+06,0.683705,5.127447,2.084040,0.928107,5.520385,0.341538,14.118468,3.390133,0.057355,0.429805,0.459513,0.495125,0.297540,0.360126,0.183981,0.430865,0.171330,2.703229,1.062190,0.962730
min,1.000000,1.000000,0.000000,4.000000,0.000000,1.000000,1.000000,7.000000,1496.000000,10.0,1.763000e+04,1.000000,21.000000,1.000000,1.000000,0.000000,0.329811,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,5.000000,750.000000,19.000000,1.000000,1.000000,42.000000,4228.000000,10.0,5.624386e+06,1.000000,21.000000,9.000000,2.000000,1.000000,0.925525,34.000000,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.550000
50%,1.000000,1.000000,11.000000,752.000000,19.000000,1.000000,1.000000,52.000000,5988.000000,10.0,8.672260e+06,1.000000,21.000000,9.000000,2.000000,2.000000,1.316269,44.000000,6.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,1.050000
75%,1.000000,1.000000,19.000000,753.000000,19.000000,2.000000,1.000000,62.000000,6319.000000,10.0,1.518998e+07,1.000000,21.000000,9.000000,2.000000,3.000000,1.392961,53.000000,9.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,1.750000
max,2.000000,9.000000,814.000000,952.000000,24.000000,4.000000,4.000000,104.000000,70601.000000,10.0,1.632872e+07,9.000000,90.000000,9.000000,9.000000,38.000000,15.517939,96.000000,12.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,19.000000,20.000000,8.700000


### Ver si el número de registro es único

In [36]:
df['numero_registro_anual'].unique()

array([8537155., 8992115., 8998349., ..., 2643973., 2573014., 2769347.],
      shape=(21209,))

In [37]:
df['centro_recodificado'].unique()

array(['-2088791444897189888', '-1166333372325380096',
       '17490445801063320188', '-3960068041784730112',
       '407388993-1955827846', '11283154571870311269', '-2517781591',
       '1749593956-1956578751', '2054924644-748389819', '-2940034890',
       '20542964201986296443', '-1280749869', '20540237801214544485',
       '-1166837180644150016', '20531759081751415396',
       '675601729962621309', '-1109365797', '981285185-580587708',
       '-924212801566861056', '675604090-111115405', '675372360794839933',
       '1480971108-1452475781', '-95835796459560992',
       '678940026-513768580', '-14682850661398800384',
       '-1471598780190870016', '-825014356', '2051359588962886267',
       '-94039200711223392', '-1295658587', '677478209945844032',
       '675822145-882872476', '9809313931634004859',
       '-1015049610897209984', '679267656-849327236',
       '675374657-278892685', '-2353644581', '11286104262038055781',
       '1128927809-478498970', '-3173934405', '-964189767951094

In [38]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21210 entries, 0 to 21209
Data columns (total 111 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    comunidad_autonoma            21210 non-null  object 
 1    fecha_nacimiento              21210 non-null  object 
 2    sexo                          21210 non-null  object 
 3    fecha_ingreso                 21210 non-null  object 
 4    circunstancia_contacto        21210 non-null  int64  
 5    fecha_fin_contacto            21210 non-null  object 
 6    tipo_alta                     21210 non-null  int64  
 7    estancia_dias                 21210 non-null  int64  
 8    diagnostico_principal         21210 non-null  object 
 9    categoria                     21210 non-null  object 
 10   diagnostico_2                 18606 non-null  object 
 11   diagnostico_3                 15060 non-null  object 
 12   diagnostico_4                 11481 non-null

### Tabla de la relación entre procedimientos y pacientes

In [39]:
# Filtrar filas con cip_sns_recodificado repetido y ordenar
conteo_cip = df['cip_sns_recodificado'].value_counts()
cips_repetidos = conteo_cip[conteo_cip > 1].index

filas_cip_repetido = df[df['cip_sns_recodificado'].isin(cips_repetidos)].sort_values('cip_sns_recodificado')
filas_cip_repetido

,comunidad_autonoma,fecha_nacimiento,sexo,fecha_ingreso,circunstancia_contacto,fecha_fin_contacto,tipo_alta,estancia_dias,diagnostico_principal,categoria,diagnostico_2,diagnostico_3,diagnostico_4,diagnostico_5,diagnostico_6,diagnostico_7,diagnostico_8,diagnostico_9,diagnostico_10,diagnostico_11,diagnostico_12,diagnostico_13,diagnostico_14,fecha_intervencion,procedimiento_1,procedimiento_2,procedimiento_3,procedimiento_4,procedimiento_5,procedimiento_6,procedimiento_7,procedimiento_8,procedimiento_9,procedimiento_10,procedimiento_11,procedimiento_12,procedimiento_13,procedimiento_14,procedimiento_15,procedimiento_16,procedimiento_17,procedimiento_18,procedimiento_19,procedimiento_20,grd_apr,cdm_apr,nivel_severidad_apr,riesgo_mortalidad_apr,servicio,edad,coste_apr,cie,numero_registro_anual,centro_recodificado,cip_sns_recodificado,pais_nacimiento,pais_residencia,fecha_inicio_contacto,regimen_financiacion,procedencia,continuidad_asistencial,ingreso_en_uci,dias_uci,diagnostico_15,diagnostico_16,diagnostico_17,diagnostico_18,diagnostico_19,diagnostico_20,poa_diagnostico_principal,poa_diagnostico_2,poa_diagnostico_3,poa_diagnostico_4,poa_diagnostico_5,poa_diagnostico_6,poa_diagnostico_7,poa_diagnostico_8,poa_diagnostico_9,poa_diagnostico_10,poa_diagnostico_11,poa_diagnostico_12,poa_diagnostico_13,poa_diagnostico_14,poa_diagnostico_15,poa_diagnostico_16,poa_diagnostico_17,poa_diagnostico_18,poa_diagnostico_19,poa_diagnostico_20,procedimiento_externo_1,procedimiento_externo_2,procedimiento_externo_3,tipo_grd_apr,peso_espanol_apr,edad_en_ingreso,mes_ingreso,tipo_alta_desc,menor_edad,nacido_espana,larga_estancia,cat_esquizofrenia,cat_trastorno_neurotico,cat_trastorno_personalidad,cat_trastorno_mental,cat_trastorno_humor,cat_trastorno_emocional,num_diagnosticos_secundarios,num_procedimientos,complejidad_general,categoria_complejidad,grupo_etario
17803,andalucia,1992-07-27,varon,2018-07-09,1,2018-07-17,1,9,F29,"Esquizofrenia, trastornos esquizotípicos y tra...",Z81.8,G44.209,E03.9,F32.2,F60.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,751,19,2,1,PSQ,33,6720,10,5741490.0,11283154571870311269,-1003994379596240000,espana,espana,2018-07-08,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,E,S,S,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.470284,25,7,domicilio,0,1,1,1,0,0,0,0,0,5,0,1.75,media,adulto temprano
13564,andalucia,1992-07-27,varon,2017-12-20,1,2018-01-03,1,14,F40.10,"Trastornos neuróticos, trastornos relacionados...",F29,F60.89,Z88.1,F23,G44.209,K58.9,M41.9,I34.1,Z81.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,755,19,2,1,PSQ,33,4948,10,5679329.0,11283154571870311269,-1003994379596240000,espana,espana,2017-12-20,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,E,S,S,S,S,S,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.082651,25,12,domicilio,0,1,1,0,1,0,0,0,0,9,0,2.95,alta,adulto temprano
16048,andalucia,1992-07-27,varon,2018-04-20,1,2018-04-30,1,10,F28,"Esquizofrenia, trastornos esquizotípicos y tra...",F06.2,F32.2,F60.89,Z88.1,G44.209,K58.9,M41.9,I34.1,Z81.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,751,19,3,1,PSQ,33,8934,10,5671719.0,11283154571870311269,-1003994379596240000,espana,espana,2018-04-20,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,E,S,S,S,S,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.954669,25,4,domicilio,0,1,1,1,0,0,0,0,0,9,0,3.20,alta,adulto temprano
21021,andalucia,1992-07-27,varon,2018-12-21,1,2018-12-28,1,8,F29,"Esquizofrenia, trastornos esquizotípicos y tra...",Z81.8,G44.209,E03.9,F60.89,F33.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,751,19,2,1,PSQ,33,6720,10,5443766.0,11283154571870311269,-1003994379596240000,espana,espana,2018-12-20,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,E,S,S,S,S,N

### Analizar si los procedimientos se repiten

In [40]:
# Filtrar las filas con CIP repetido que tienen procedimiento_1 no nulo
filas_con_procedimiento = filas_cip_repetido[filas_cip_repetido['procedimiento_1'].notna()]
filas_con_procedimiento

,comunidad_autonoma,fecha_nacimiento,sexo,fecha_ingreso,circunstancia_contacto,fecha_fin_contacto,tipo_alta,estancia_dias,diagnostico_principal,categoria,diagnostico_2,diagnostico_3,diagnostico_4,diagnostico_5,diagnostico_6,diagnostico_7,diagnostico_8,diagnostico_9,diagnostico_10,diagnostico_11,diagnostico_12,diagnostico_13,diagnostico_14,fecha_intervencion,procedimiento_1,procedimiento_2,procedimiento_3,procedimiento_4,procedimiento_5,procedimiento_6,procedimiento_7,procedimiento_8,procedimiento_9,procedimiento_10,procedimiento_11,procedimiento_12,procedimiento_13,procedimiento_14,procedimiento_15,procedimiento_16,procedimiento_17,procedimiento_18,procedimiento_19,procedimiento_20,grd_apr,cdm_apr,nivel_severidad_apr,riesgo_mortalidad_apr,servicio,edad,coste_apr,cie,numero_registro_anual,centro_recodificado,cip_sns_recodificado,pais_nacimiento,pais_residencia,fecha_inicio_contacto,regimen_financiacion,procedencia,continuidad_asistencial,ingreso_en_uci,dias_uci,diagnostico_15,diagnostico_16,diagnostico_17,diagnostico_18,diagnostico_19,diagnostico_20,poa_diagnostico_principal,poa_diagnostico_2,poa_diagnostico_3,poa_diagnostico_4,poa_diagnostico_5,poa_diagnostico_6,poa_diagnostico_7,poa_diagnostico_8,poa_diagnostico_9,poa_diagnostico_10,poa_diagnostico_11,poa_diagnostico_12,poa_diagnostico_13,poa_diagnostico_14,poa_diagnostico_15,poa_diagnostico_16,poa_diagnostico_17,poa_diagnostico_18,poa_diagnostico_19,poa_diagnostico_20,procedimiento_externo_1,procedimiento_externo_2,procedimiento_externo_3,tipo_grd_apr,peso_espanol_apr,edad_en_ingreso,mes_ingreso,tipo_alta_desc,menor_edad,nacido_espana,larga_estancia,cat_esquizofrenia,cat_trastorno_neurotico,cat_trastorno_personalidad,cat_trastorno_mental,cat_trastorno_humor,cat_trastorno_emocional,num_diagnosticos_secundarios,num_procedimientos,complejidad_general,categoria_complejidad,grupo_etario
2954,andalucia,1943-11-07,varon,2016-05-26,1,2016-05-31,1,5,F29,"Esquizofrenia, trastornos esquizotípicos y tra...",F32.9,F51.05,E11.9,I50.9,I10,I25.5,I25.2,E78.0,N40.0,E87.1,Z87.891,Z60.2,NaN,NaN,B020ZZZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,751,19,3,2,PSQ,81,7263,10,8912499.0,2054924644-748389819,-10049380031552200704,desconocido,espana,2016-05-26,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,S,S,E,S,S,S,E,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.596424,72,5,domicilio,0,0,0,1,0,0,0,0,0,12,1,4.55,muy alta,geriatrico
19034,andalucia,1976-12-26,mujer,2018-09-08,1,2018-10-15,1,37,F20.0,"Esquizofrenia, trastornos esquizotípicos y tra...",G40.509,R00.0,T42.4X5A,F17.210,J45.909,E11.9,Z81.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B020ZZZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,2,1,PSQ,48,7362,10,5437733.0,-3960068041784730112,-10089848531585599488,espana,espana,2018-09-08,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,N,N,S,S,S,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.610675,41,9,domicilio,0,1,1,1,0,0,0,0,0,7,1,2.55,alta,adulto medio
8636,andalucia,1976-12-26,mujer,2017-03-24,1,2017-04-25,1,32,F20.0,"Esquizofrenia, trastornos esquizotípicos y tra...",J44.9,E11.65,Z81.8,Z91.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,B020ZZZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,2,1,PSQ,48,6319,10,15726297.0,-3960068041784730112,-10089848531585599488,espana,espana,2017-03-24,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,E,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.392961,40,3,domicilio,0,1,1,1,0,0,0,0,0,4,1,1.65,media,adulto medio
18623,andalucia,1962-11-04,mujer,2018-08-17,1,2018-08-28,1,11,F91.8,Trastornos emocionales y del comportamiento qu...,F19.159,F10.14,F41.8,F51.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GZ56ZZZ,GZ72ZZZ,F08Z6ZZ,GZ3ZZZZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,758,19,1,1,PSQ,62,3393,10,5349062.0,-3173934405,-1010549763797159936,espana,espana,2018-08-17,1.0,21.0,9.0,2.0,NaN,NaN,NaN,Na

Creación de la tabla de muchos a muchos entre procedimientos y pacientes:

In [41]:
# Crear tabla df_procedimientos_pacientes actualizada incluyendo procedimientos externos
print("CREANDO TABLA DE PROCEDIMIENTOS COMPLETA (incluyendo externos)")
print("="*65)

# Identificar todas las columnas de procedimientos (internos y externos)
columnas_procedimientos_internos = [col for col in df.columns if col.startswith('procedimiento_') and not col.startswith('procedimiento_externo')]
columnas_procedimientos_externos = [col for col in df.columns if col.startswith('procedimiento_externo_')]

print(f"Columnas de procedimientos internos: {len(columnas_procedimientos_internos)}")
for col in columnas_procedimientos_internos:
    print(f"  - {col}")

print(f"\nColumnas de procedimientos externos: {len(columnas_procedimientos_externos)}")
for col in columnas_procedimientos_externos:
    print(f"  - {col}")

# Crear lista para almacenar los datos de la tabla normalizada
datos_procedimientos = []

# Procesar procedimientos internos
for index, row in df.iterrows():
    cip_paciente = row['cip_sns_recodificado']
    
    # Recorrer procedimientos internos
    for col_proc in columnas_procedimientos_internos:
        valor_procedimiento = row[col_proc]
        
        if pd.notna(valor_procedimiento) and valor_procedimiento != '':
            numero_procedimiento = int(col_proc.split('_')[1])
            
            datos_procedimientos.append({
                'cip_sns_recodificado': cip_paciente,
                'codigo_procedimiento': valor_procedimiento,
                'numero_procedimiento': numero_procedimiento,
                'es_externo': 0
            })
    
    # Recorrer procedimientos externos
    for col_proc in columnas_procedimientos_externos:
        valor_procedimiento = row[col_proc]
        
        if pd.notna(valor_procedimiento) and valor_procedimiento != '':
            numero_procedimiento = int(col_proc.split('_')[2])  # procedimiento_externo_1 -> 1
            
            datos_procedimientos.append({
                'cip_sns_recodificado': cip_paciente,
                'codigo_procedimiento': valor_procedimiento,
                'numero_procedimiento': numero_procedimiento,
                'es_externo': 1
            })

# Crear el DataFrame actualizado
df_procedimientos_pacientes = pd.DataFrame(datos_procedimientos)

# Eliminar duplicados
df_procedimientos_pacientes = df_procedimientos_pacientes.drop_duplicates()

print(f"\nTabla df_procedimientos_pacientes actualizada creada:")
print(f"  - Filas totales: {len(df_procedimientos_pacientes)}")
print(f"  - Pacientes únicos: {df_procedimientos_pacientes['cip_sns_recodificado'].nunique()}")
print(f"  - Procedimientos únicos: {df_procedimientos_pacientes['codigo_procedimiento'].nunique()}")
print(f"  - Procedimientos internos: {len(df_procedimientos_pacientes[df_procedimientos_pacientes['es_externo'] == 0])}")
print(f"  - Procedimientos externos: {len(df_procedimientos_pacientes[df_procedimientos_pacientes['es_externo'] == 1])}")

print(f"\nPrimeras 10 filas:")
print(df_procedimientos_pacientes.head(10))

print(f"\nDistribución por tipo de procedimiento:")
print(df_procedimientos_pacientes['es_externo'].value_counts())

df_procedimientos_pacientes

CREANDO TABLA DE PROCEDIMIENTOS COMPLETA (incluyendo externos)
Columnas de procedimientos internos: 20
  - procedimiento_1
  - procedimiento_2
  - procedimiento_3
  - procedimiento_4
  - procedimiento_5
  - procedimiento_6
  - procedimiento_7
  - procedimiento_8
  - procedimiento_9
  - procedimiento_10
  - procedimiento_11
  - procedimiento_12
  - procedimiento_13
  - procedimiento_14
  - procedimiento_15
  - procedimiento_16
  - procedimiento_17
  - procedimiento_18
  - procedimiento_19
  - procedimiento_20

Columnas de procedimientos externos: 3
  - procedimiento_externo_1
  - procedimiento_externo_2
  - procedimiento_externo_3

Tabla df_procedimientos_pacientes actualizada creada:
  - Filas totales: 8076
  - Pacientes únicos: 3344
  - Procedimientos únicos: 597
  - Procedimientos internos: 8071
  - Procedimientos externos: 5

Primeras 10 filas:
   cip_sns_recodificado codigo_procedimiento  numero_procedimiento  es_externo
0  -1589750168781380096              4B02XSZ                 

,cip_sns_recodificado,codigo_procedimiento,numero_procedimiento,es_externo
0,-1589750168781380096,4B02XSZ,1,0
1,-1589750168781380096,B246ZZZ,2,0
2,-1589750168781380096,4A02X4Z,3,0
3,18237511542021901538,GZ3ZZZZ,1,0
4,-1928562448814109952,GZ14ZZZ,1,0
...,...,...,...,...
9391,1332765377951591068,B020ZZZ,1,0
9392,576616051-23118435,GZ3ZZZZ,1,0
9393,-2853188222,4A033R1,1,0
9394,-2853188222,3E0337Z,2,0


### Creación de la tabla de procedimientos

In [42]:
# Leer el JSON de procedimientos y crear DataFrame
import json

# Cargar el JSON de procedimientos
with open('../jsons/procedimientos.json', 'r', encoding='utf-8') as f:
    datos_procedimientos_json = json.load(f)

# El JSON tiene formato {codigo: descripcion}, necesitamos convertirlo a DataFrame
# Crear listas para ID y descripción
codigos = list(datos_procedimientos_json.keys())
descripciones = list(datos_procedimientos_json.values())

# Crear DataFrame con las columnas ID y descripción
df_procedimientos = pd.DataFrame({
    'ID': codigos,
    'descripcion': descripciones
})

print(f"Tabla df_procedimientos creada:")
print(f"  - Total de procedimientos: {len(df_procedimientos)}")
print(f"\nPrimeras 10 filas:")
print(df_procedimientos.head(10))

df_procedimientos

Tabla df_procedimientos creada:
  - Total de procedimientos: 597

Primeras 10 filas:
        ID                                        descripcion
0  009500Z  Drenaje de espacio subaracnoideo lumbar, abord...
1  009U30Z     Drenaje de médula espinal, abordaje percutáneo
2  009U3ZX  Drenaje de médula espinal, abordaje percutáneo...
3  009U3ZZ  Drenaje de médula espinal, abordaje percutáneo...
4  009Y3ZX  Drenaje de médula espinal y nervio espinal, pe...
5  009Y3ZZ  Drenaje de médula espinal y nervio espinal, pe...
6  00H03MZ  Inserción de estimulador en cerebro, abordaje ...
7  00HU3MZ  Inserción de estimulador en médula espinal, ab...
8  00NK3ZZ  Liberación de nervio craneal, abordaje percutáneo
9  02583ZZ  Destrucción de conducción cardíaca, abordaje p...


,ID,descripcion
0,009500Z,"Drenaje de espacio subaracnoideo lumbar, abord..."
1,009U30Z,"Drenaje de médula espinal, abordaje percutáneo"
2,009U3ZX,"Drenaje de médula espinal, abordaje percutáneo..."
3,009U3ZZ,"Drenaje de médula espinal, abordaje percutáneo..."
4,009Y3ZX,"Drenaje de médula espinal y nervio espinal, pe..."
...,...,...
592,HZ97ZZZ,HZ97ZZZ - Tratamiento de Abuso de Sustancias
593,HZ98ZZZ,HZ98ZZZ - Tratamiento de Abuso de Sustancias
594,HZ99ZZZ,HZ99ZZZ - Tratamiento de Abuso de Sustancias
595,ZZZZZZP,ZZZZZZP - Sección desconocida (Z)


### Creación de la tabla de diagnósticos:

In [43]:
# Leer el JSON de diagnósticos y crear DataFrame
# Cargar el JSON de diagnósticos
with open('../jsons/diagnosticos.json', 'r', encoding='utf-8') as f:
    datos_diagnosticos_json = json.load(f)

# El JSON tiene formato {codigo: descripcion}, necesitamos convertirlo a DataFrame
# Crear listas para ID y descripción
codigos_diagnosticos = list(datos_diagnosticos_json.keys())
descripciones_diagnosticos = list(datos_diagnosticos_json.values())

# Crear DataFrame con las columnas ID y descripción
df_diagnosticos = pd.DataFrame({
    'ID': codigos_diagnosticos,
    'descripcion': descripciones_diagnosticos
})

print(f"Tabla df_diagnosticos creada:")
print(f"  - Total de diagnósticos: {len(df_diagnosticos)}")
print(f"\nPrimeras 10 filas:")
print(df_diagnosticos.head(10))

df_diagnosticos

Tabla df_diagnosticos creada:
  - Total de diagnósticos: 381

Primeras 10 filas:
       ID                     descripcion
0   F20.0         Esquizofrenia paranoide
1   F20.1       Esquizofrenia hebefrenica
2   F20.2        Esquizofrenia catatónica
3   F20.3    Esquizofrenia indiferenciada
4   F20.5          Esquizofrenia residual
5  F20.81     Trastorno esquizofreniforme
6  F20.89            Otras esquizofrenias
7   F20.9  Esquizofrenia, no especificada
8     F21         Trastorno esquizotípico
9     F24  Trastorno psicótico compartido


,ID,descripcion
0,F20.0,Esquizofrenia paranoide
1,F20.1,Esquizofrenia hebefrenica
2,F20.2,Esquizofrenia catatónica
3,F20.3,Esquizofrenia indiferenciada
4,F20.5,Esquizofrenia residual
...,...,...
376,T43.595S,Efecto adverso de otros antipsicóticos y neuro...
377,T36.8X5A,Efecto adverso de otros antibióticos sistémico...
378,S29.9XXA,"Lesión no especificada del tórax, encuentro in..."
379,X58.XXXA,"Exposición a otros factores especificados, enc..."


In [44]:
df.head()

,comunidad_autonoma,fecha_nacimiento,sexo,fecha_ingreso,circunstancia_contacto,fecha_fin_contacto,tipo_alta,estancia_dias,diagnostico_principal,categoria,diagnostico_2,diagnostico_3,diagnostico_4,diagnostico_5,diagnostico_6,diagnostico_7,diagnostico_8,diagnostico_9,diagnostico_10,diagnostico_11,diagnostico_12,diagnostico_13,diagnostico_14,fecha_intervencion,procedimiento_1,procedimiento_2,procedimiento_3,procedimiento_4,procedimiento_5,procedimiento_6,procedimiento_7,procedimiento_8,procedimiento_9,procedimiento_10,procedimiento_11,procedimiento_12,procedimiento_13,procedimiento_14,procedimiento_15,procedimiento_16,procedimiento_17,procedimiento_18,procedimiento_19,procedimiento_20,grd_apr,cdm_apr,nivel_severidad_apr,riesgo_mortalidad_apr,servicio,edad,coste_apr,cie,numero_registro_anual,centro_recodificado,cip_sns_recodificado,pais_nacimiento,pais_residencia,fecha_inicio_contacto,regimen_financiacion,procedencia,continuidad_asistencial,ingreso_en_uci,dias_uci,diagnostico_15,diagnostico_16,diagnostico_17,diagnostico_18,diagnostico_19,diagnostico_20,poa_diagnostico_principal,poa_diagnostico_2,poa_diagnostico_3,poa_diagnostico_4,poa_diagnostico_5,poa_diagnostico_6,poa_diagnostico_7,poa_diagnostico_8,poa_diagnostico_9,poa_diagnostico_10,poa_diagnostico_11,poa_diagnostico_12,poa_diagnostico_13,poa_diagnostico_14,poa_diagnostico_15,poa_diagnostico_16,poa_diagnostico_17,poa_diagnostico_18,poa_diagnostico_19,poa_diagnostico_20,procedimiento_externo_1,procedimiento_externo_2,procedimiento_externo_3,tipo_grd_apr,peso_espanol_apr,edad_en_ingreso,mes_ingreso,tipo_alta_desc,menor_edad,nacido_espana,larga_estancia,cat_esquizofrenia,cat_trastorno_neurotico,cat_trastorno_personalidad,cat_trastorno_mental,cat_trastorno_humor,cat_trastorno_emocional,num_diagnosticos_secundarios,num_procedimientos,complejidad_general,categoria_complejidad,grupo_etario
0,andalucia,1951-08-17,mujer,2016-01-01,1,2016-01-08,1,7,F25.0,"Esquizofrenia, trastornos esquizotípicos y tra...",Z63.79,Z91.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,2,1,PSQ,74,6340,10,8537155.0,-2088791444897189888,109457269-593755146,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,E,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,1.393611,64,1,domicilio,0,1,1,1,0,0,0,0,0,2,0,0.85,baja,adulto mayor
1,andalucia,1929-03-20,mujer,2016-01-01,1,2016-01-08,1,7,F41.9,"Trastornos neuróticos, trastornos relacionados...",I11.9,I35.8,E11.9,I87.2,Z95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4B02XSZ,B246ZZZ,4A02X4Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,756,19,1,2,CAR,96,2771,10,8992115.0,-1166333372325380096,-1589750168781380096,desconocido,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.609264,86,1,domicilio,0,0,1,0,1,0,0,0,0,5,3,2.35,media,geriatrico
2,andalucia,1976-11-25,varon,2016-01-01,1,2016-01-11,1,10,F60.2,Trastornos de la personalidad y del comportami...,F19.288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,752,19,2,1,PSQ,48,4009,10,8998349.0,17490445801063320188,-5406560181117020160,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M,0.881297,39,1,domicilio,0,1,1,0,0,1,0,0,0,1,0,0.55,baja,adulto medio
3,andalucia,1976-11-10,mujer,2016-01-01,1,2016-01-27,1,26,F20.0,"Esquizofrenia, trastornos esquizotípicos y tra...",C07,F17.210,F12.20,F14.10,F10.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,750,19,1,2,PSQ,48,6073,10,8800205.0,-3960068041784730112,-1823171082,espana,espana,2016-01-01,1.0,21.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,S,S,S,S,S,NaN,NaN,NaN,NaN,NaN,

### Creación de la relación entre diagnósticos y pacientes

In [45]:
# Crear tabla df_diagnosticos_pacientes para relación muchos a muchos
print("CREANDO TABLA DE DIAGNÓSTICOS-PACIENTES")
print("="*50)

# Identificar todas las columnas de diagnósticos y poa_diagnosticos
columnas_diagnosticos = [col for col in df.columns if col.startswith('diagnostico_')]
columnas_poa_diagnosticos = [col for col in df.columns if col.startswith('poa_diagnostico_')]

print(f"Columnas de diagnósticos encontradas: {len(columnas_diagnosticos)}")
for col in columnas_diagnosticos:
    print(f"  - {col}")

print(f"\nColumnas de POA diagnósticos encontradas: {len(columnas_poa_diagnosticos)}")
for col in columnas_poa_diagnosticos:
    print(f"  - {col}")

# Crear lista para almacenar los datos de la tabla normalizada
datos_diagnosticos_pacientes = []

# Recorrer cada fila del dataframe original
for index, row in df.iterrows():
    cip_paciente = row['cip_sns_recodificado']
    
    # Procesar diagnóstico principal
    if pd.notna(row['diagnostico_principal']) and row['diagnostico_principal'] != '':
        # Buscar el POA correspondiente al diagnóstico principal
        poa_valor = row.get('poa_diagnostico_principal', None)
        
        datos_diagnosticos_pacientes.append({
            'cip_sns_recodificado': cip_paciente,
            'codigo_diagnostico': row['diagnostico_principal'],
            'diagnostico_principal': 1,
            'poa_diagnostico': poa_valor
        })
    
    # Procesar diagnósticos secundarios (1 al N)
    for col_diag in columnas_diagnosticos:
        if col_diag != 'diagnostico_principal':  # Excluir el principal ya procesado
            valor_diagnostico = row[col_diag]
            
            if pd.notna(valor_diagnostico) and valor_diagnostico != '':
                # Extraer el número del diagnóstico para buscar el POA correspondiente
                # diagnostico_1 -> 1, diagnostico_2 -> 2, etc.
                numero_diagnostico = col_diag.split('_')[1]
                col_poa_correspondiente = f'poa_diagnostico_{numero_diagnostico}'
                
                # Buscar el POA correspondiente
                poa_valor = row.get(col_poa_correspondiente, None)
                
                datos_diagnosticos_pacientes.append({
                    'cip_sns_recodificado': cip_paciente,
                    'codigo_diagnostico': valor_diagnostico,
                    'diagnostico_principal': 0,
                    'poa_diagnostico': poa_valor
                })

# Crear el DataFrame
df_diagnosticos_pacientes = pd.DataFrame(datos_diagnosticos_pacientes)

# Eliminar duplicados
df_diagnosticos_pacientes = df_diagnosticos_pacientes.drop_duplicates()

print(f"\nTabla df_diagnosticos_pacientes creada:")
print(f"  - Filas totales: {len(df_diagnosticos_pacientes)}")
print(f"  - Pacientes únicos: {df_diagnosticos_pacientes['cip_sns_recodificado'].nunique()}")
print(f"  - Diagnósticos únicos: {df_diagnosticos_pacientes['codigo_diagnostico'].nunique()}")
print(f"  - Diagnósticos principales: {len(df_diagnosticos_pacientes[df_diagnosticos_pacientes['diagnostico_principal'] == 1])}")
print(f"  - Diagnósticos secundarios: {len(df_diagnosticos_pacientes[df_diagnosticos_pacientes['diagnostico_principal'] == 0])}")

print(f"\nDistribución por tipo de diagnóstico:")
print(df_diagnosticos_pacientes['diagnostico_principal'].value_counts())

print(f"\nDistribución de valores POA:")
print(df_diagnosticos_pacientes['poa_diagnostico'].value_counts(dropna=False))

print(f"\nPrimeras 10 filas:")
print(df_diagnosticos_pacientes.head(10))

df_diagnosticos_pacientes

CREANDO TABLA DE DIAGNÓSTICOS-PACIENTES
Columnas de diagnósticos encontradas: 20
  - diagnostico_principal
  - diagnostico_2
  - diagnostico_3
  - diagnostico_4
  - diagnostico_5
  - diagnostico_6
  - diagnostico_7
  - diagnostico_8
  - diagnostico_9
  - diagnostico_10
  - diagnostico_11
  - diagnostico_12
  - diagnostico_13
  - diagnostico_14
  - diagnostico_15
  - diagnostico_16
  - diagnostico_17
  - diagnostico_18
  - diagnostico_19
  - diagnostico_20

Columnas de POA diagnósticos encontradas: 20
  - poa_diagnostico_principal
  - poa_diagnostico_2
  - poa_diagnostico_3
  - poa_diagnostico_4
  - poa_diagnostico_5
  - poa_diagnostico_6
  - poa_diagnostico_7
  - poa_diagnostico_8
  - poa_diagnostico_9
  - poa_diagnostico_10
  - poa_diagnostico_11
  - poa_diagnostico_12
  - poa_diagnostico_13
  - poa_diagnostico_14
  - poa_diagnostico_15
  - poa_diagnostico_16
  - poa_diagnostico_17
  - poa_diagnostico_18
  - poa_diagnostico_19
  - poa_diagnostico_20

Tabla df_diagnosticos_pacientes cr

,cip_sns_recodificado,codigo_diagnostico,diagnostico_principal,poa_diagnostico
0,109457269-593755146,F25.0,1,S
1,109457269-593755146,Z63.79,0,E
2,109457269-593755146,Z91.19,0,S
3,-1589750168781380096,F41.9,1,S
4,-1589750168781380096,I11.9,0,S
...,...,...,...,...
91144,1160647917-1937584955,F17.210,0,S
91145,1160647917-1937584955,F10.20,0,S
91146,1160647917-1937584955,F14.21,0,S
91155,3583339492008637068,F19.122,0,S


In [46]:
# Análisis de columnas: Paciente vs Ingreso/Episodio
print("CLASIFICACIÓN DE COLUMNAS: PACIENTE vs INGRESO")
print("="*60)

# Columnas del PACIENTE (información demográfica/personal que no cambia)
columnas_paciente = [
    'cip_sns_recodificado',  # ID único del paciente
    'fecha_nacimiento',      # Fecha de nacimiento
    'sexo',                  # Sexo del paciente
    'pais_nacimiento',       # País de nacimiento
    'pais_residencia',       # País de residencia
    'edad',                  # Edad del paciente
    'edad_en_ingreso',       # Edad específica en este ingreso
    'menor_edad',            # Derivada de edad
    'nacido_espana',         # Derivada de país nacimiento
    'grupo_etario'           # Derivada de edad
]

# Columnas del INGRESO/EPISODIO (información específica de cada hospitalización)
columnas_ingreso = [
    'numero_registro_anual',      # ID del episodio
    'comunidad_autonoma',         # Dónde ocurrió el ingreso
    'centro_recodificado',        # Hospital del ingreso
    'fecha_ingreso',              # Fecha de ingreso
    'fecha_inicio_contacto',      # Inicio del contacto
    'fecha_fin_contacto',         # Fin del contacto
    'fecha_intervencion',         # Fecha de intervención
    'circunstancia_contacto',     # Cómo llegó al hospital
    'tipo_alta',                  # Tipo de alta
    'tipo_alta_desc',             # Descripción del alta
    'estancia_dias',              # Duración estancia
    'larga_estancia',             # Derivada de estancia
    'servicio',                   # Servicio hospitalario
    'regimen_financiacion',       # Régimen financiero
    'procedencia',                # De dónde viene
    'continuidad_asistencial',    # Continuidad
    'ingreso_en_uci',            # Si ingresó en UCI
    'dias_uci',                  # Días en UCI
    'mes_ingreso',               # Mes del ingreso
    'categoria',                 # Categoría diagnóstica del episodio
    'grd_apr',                   # GRD de este episodio
    'cdm_apr',                   # CDM de este episodio
    'nivel_severidad_apr',       # Severidad en este episodio
    'riesgo_mortalidad_apr',     # Riesgo mortalidad este episodio
    'tipo_grd_apr',              # Tipo GRD
    'peso_espanol_apr',          # Peso español APR
    'coste_apr',                 # Costo de este episodio
    'cie',                       # Clasificación CIE
    'complejidad_general',       # Complejidad de este episodio
    'categoria_complejidad',     # Categoría complejidad
    'num_diagnosticos_secundarios', # Conteo para este episodio
    'num_procedimientos',        # Conteo para este episodio
    # Columnas dummy de categorías (específicas del episodio)
    'cat_esquizofrenia',
    'cat_trastorno_neurotico',
    'cat_trastorno_personalidad',
    'cat_trastorno_mental',
    'cat_trastorno_humor',
    'cat_trastorno_emocional'
]

# Columnas de DIAGNÓSTICOS Y PROCEDIMIENTOS (se normalizarán por separado)
columnas_diagnosticos_proc = []
for col in df.columns:
    if (col.startswith('diagnostico_') or 
        col.startswith('poa_diagnostico_') or 
        col.startswith('procedimiento_')):
        columnas_diagnosticos_proc.append(col)

print(f"COLUMNAS DEL PACIENTE ({len(columnas_paciente)}):")
for col in columnas_paciente:
    print(f"  - {col}")

print(f"\nCOLUMNAS DEL INGRESO/EPISODIO ({len(columnas_ingreso)}):")
for col in columnas_ingreso:
    print(f"  - {col}")

print(f"\nCOLUMNAS DE DIAGNÓSTICOS/PROCEDIMIENTOS ({len(columnas_diagnosticos_proc)}):")
for i, col in enumerate(columnas_diagnosticos_proc):
    if i < 10:  # Mostrar solo las primeras 10
        print(f"  - {col}")
    elif i == 10:
        print(f"  ... y {len(columnas_diagnosticos_proc)-10} más")

# Verificar que todas las columnas están clasificadas
todas_columnas = set(df.columns)
clasificadas = set(columnas_paciente + columnas_ingreso + columnas_diagnosticos_proc)
no_clasificadas = todas_columnas - clasificadas

if no_clasificadas:
    print(f"\nCOLUMNAS NO CLASIFICADAS ({len(no_clasificadas)}):")
    for col in no_clasificadas:
        print(f"  - {col}")
else:
    print(f"\n✓ Todas las columnas han sido clasificadas correctamente")

print(f"\nRESUMEN:")
print(f"  - Paciente: {len(columnas_paciente)} columnas")
print(f"  - Ingreso: {len(columnas_ingreso)} columnas") 
print(f"  - Diagnósticos/Procedimientos: {len(columnas_diagnosticos_proc)} columnas")
print(f"  - Total: {len(clasificadas)} de {len(todas_columnas)} columnas")

CLASIFICACIÓN DE COLUMNAS: PACIENTE vs INGRESO
COLUMNAS DEL PACIENTE (10):
  - cip_sns_recodificado
  - fecha_nacimiento
  - sexo
  - pais_nacimiento
  - pais_residencia
  - edad
  - edad_en_ingreso
  - menor_edad
  - nacido_espana
  - grupo_etario

COLUMNAS DEL INGRESO/EPISODIO (38):
  - numero_registro_anual
  - comunidad_autonoma
  - centro_recodificado
  - fecha_ingreso
  - fecha_inicio_contacto
  - fecha_fin_contacto
  - fecha_intervencion
  - circunstancia_contacto
  - tipo_alta
  - tipo_alta_desc
  - estancia_dias
  - larga_estancia
  - servicio
  - regimen_financiacion
  - procedencia
  - continuidad_asistencial
  - ingreso_en_uci
  - dias_uci
  - mes_ingreso
  - categoria
  - grd_apr
  - cdm_apr
  - nivel_severidad_apr
  - riesgo_mortalidad_apr
  - tipo_grd_apr
  - peso_espanol_apr
  - coste_apr
  - cie
  - complejidad_general
  - categoria_complejidad
  - num_diagnosticos_secundarios
  - num_procedimientos
  - cat_esquizofrenia
  - cat_trastorno_neurotico
  - cat_trastorno_pe

### Creación de las tablas normalizadas: Pacientes e Ingresos

In [47]:
# Crear tabla de Pacientes (df_pacientes)
print("CREANDO TABLA DE PACIENTES")
print("="*40)

# Crear DataFrame de pacientes con columnas únicas por paciente
df_pacientes = df[columnas_paciente].drop_duplicates(subset=['cip_sns_recodificado']).copy()

print(f"Tabla df_pacientes creada:")
print(f"  - Pacientes únicos: {len(df_pacientes)}")
print(f"  - Columnas: {len(df_pacientes.columns)}")

# Verificar que no hay duplicados por cip_sns_recodificado
duplicados_pacientes = df_pacientes.duplicated(subset=['cip_sns_recodificado']).sum()
if duplicados_pacientes == 0:
    print(f"  ✓ Sin duplicados en cip_sns_recodificado")
else:
    print(f"  ⚠️ {duplicados_pacientes} duplicados encontrados en cip_sns_recodificado")

print(f"\nPrimeras 5 filas de df_pacientes:")
print(df_pacientes.head())

print(f"\nInformación de df_pacientes:")
print(df_pacientes.info())

df_pacientes

CREANDO TABLA DE PACIENTES
Tabla df_pacientes creada:
  - Pacientes únicos: 11875
  - Columnas: 10
  ✓ Sin duplicados en cip_sns_recodificado

Primeras 5 filas de df_pacientes:
   cip_sns_recodificado fecha_nacimiento   sexo pais_nacimiento  \
0   109457269-593755146       1951-08-17  mujer          espana   
1  -1589750168781380096       1929-03-20  mujer     desconocido   
2  -5406560181117020160       1976-11-25  varon          espana   
3           -1823171082       1976-11-10  mujer          espana   
4           -2828047377       1977-04-28  mujer          espana   

  pais_residencia  edad  edad_en_ingreso  menor_edad  nacido_espana  \
0          espana    74               64           0              1   
1          espana    96               86           0              0   
2          espana    48               39           0              1   
3          espana    48               39           0              1   
4          espana    48               38           0             

,cip_sns_recodificado,fecha_nacimiento,sexo,pais_nacimiento,pais_residencia,edad,edad_en_ingreso,menor_edad,nacido_espana,grupo_etario
0,109457269-593755146,1951-08-17,mujer,espana,espana,74,64,0,1,adulto mayor
1,-1589750168781380096,1929-03-20,mujer,desconocido,espana,96,86,0,0,geriatrico
2,-5406560181117020160,1976-11-25,varon,espana,espana,48,39,0,1,adulto medio
3,-1823171082,1976-11-10,mujer,espana,espana,48,39,0,1,adulto medio
4,-2828047377,1977-04-28,mujer,espana,espana,48,38,0,1,adulto medio
...,...,...,...,...,...,...,...,...,...,...
21198,-12910484591673100288,1954-03-31,varon,espana,espana,71,64,0,1,adulto mayor
21199,-2853188222,2005-02-20,varon,espana,espana,20,13,0,1,joven adulto
21201,-2675981073,1961-02-12,varon,espana,espana,64,57,0,1,adulto mayor
21202,1871398775-2070053134,1932-05-16,mujer,espana,espana,93,86,0,1,geriatrico


In [48]:
# Crear tabla de Ingresos (df_ingresos)
print("CREANDO TABLA DE INGRESOS")
print("="*40)

# Crear DataFrame de ingresos con todas las filas (cada fila es un ingreso)
# Incluir cip_sns_recodificado como clave foránea al paciente
columnas_ingreso_completas = ['cip_sns_recodificado'] + columnas_ingreso

df_ingresos = df[columnas_ingreso_completas].copy()

print(f"Tabla df_ingresos creada:")
print(f"  - Total de ingresos: {len(df_ingresos)}")
print(f"  - Columnas: {len(df_ingresos.columns)}")
print(f"  - Pacientes únicos: {df_ingresos['cip_sns_recodificado'].nunique()}")

# Verificar que numero_registro_anual es único (cada fila es un ingreso único)
duplicados_ingresos = df_ingresos.duplicated(subset=['numero_registro_anual']).sum()
if duplicados_ingresos == 0:
    print(f"  ✓ Sin duplicados en numero_registro_anual")
else:
    print(f"  ⚠️ {duplicados_ingresos} duplicados encontrados en numero_registro_anual")

# Estadísticas de ingresos por paciente
ingresos_por_paciente = df_ingresos['cip_sns_recodificado'].value_counts()
print(f"\nEstadísticas de ingresos por paciente:")
print(f"  - Promedio: {ingresos_por_paciente.mean():.2f} ingresos/paciente")
print(f"  - Mediana: {ingresos_por_paciente.median():.1f} ingresos/paciente")
print(f"  - Máximo: {ingresos_por_paciente.max()} ingresos/paciente")
print(f"  - Pacientes con múltiples ingresos: {(ingresos_por_paciente > 1).sum()}")

print(f"\nDistribución de número de ingresos:")
distribucion_ingresos = ingresos_por_paciente.value_counts().head(10)
for num_ingresos, cantidad_pacientes in distribucion_ingresos.items():
    print(f"  - {cantidad_pacientes} pacientes con {num_ingresos} ingreso(s)")

print(f"\nPrimeras 5 filas de df_ingresos:")
print(df_ingresos.head())

print(f"\nInformación de df_ingresos:")
print(df_ingresos.info())

df_ingresos

CREANDO TABLA DE INGRESOS
Tabla df_ingresos creada:
  - Total de ingresos: 21210
  - Columnas: 39
  - Pacientes únicos: 11874
  ⚠️ 1 duplicados encontrados en numero_registro_anual

Estadísticas de ingresos por paciente:
  - Promedio: 1.71 ingresos/paciente
  - Mediana: 1.0 ingresos/paciente
  - Máximo: 29 ingresos/paciente
  - Pacientes con múltiples ingresos: 3762

Distribución de número de ingresos:
  - 8112 pacientes con 1 ingreso(s)
  - 1962 pacientes con 2 ingreso(s)
  - 799 pacientes con 3 ingreso(s)
  - 392 pacientes con 4 ingreso(s)
  - 226 pacientes con 5 ingreso(s)
  - 117 pacientes con 6 ingreso(s)
  - 96 pacientes con 7 ingreso(s)
  - 57 pacientes con 8 ingreso(s)
  - 35 pacientes con 9 ingreso(s)
  - 20 pacientes con 10 ingreso(s)

Primeras 5 filas de df_ingresos:
   cip_sns_recodificado  numero_registro_anual comunidad_autonoma  \
0   109457269-593755146              8537155.0          andalucia   
1  -1589750168781380096              8992115.0          andalucia   
2  -

,cip_sns_recodificado,numero_registro_anual,comunidad_autonoma,centro_recodificado,fecha_ingreso,fecha_inicio_contacto,fecha_fin_contacto,fecha_intervencion,circunstancia_contacto,tipo_alta,tipo_alta_desc,estancia_dias,larga_estancia,servicio,regimen_financiacion,procedencia,continuidad_asistencial,ingreso_en_uci,dias_uci,mes_ingreso,categoria,grd_apr,cdm_apr,nivel_severidad_apr,riesgo_mortalidad_apr,tipo_grd_apr,peso_espanol_apr,coste_apr,cie,complejidad_general,categoria_complejidad,num_diagnosticos_secundarios,num_procedimientos,cat_esquizofrenia,cat_trastorno_neurotico,cat_trastorno_personalidad,cat_trastorno_mental,cat_trastorno_humor,cat_trastorno_emocional
0,109457269-593755146,8537155.0,andalucia,-2088791444897189888,2016-01-01,2016-01-01,2016-01-08,NaN,1,1,domicilio,7,1,PSQ,1.0,21.0,9.0,2.0,NaN,1,"Esquizofrenia, trastornos esquizotípicos y tra...",750,19,2,1,M,1.393611,6340,10,0.85,baja,2,0,1,0,0,0,0,0
1,-1589750168781380096,8992115.0,andalucia,-1166333372325380096,2016-01-01,2016-01-01,2016-01-08,NaN,1,1,domicilio,7,1,CAR,1.0,21.0,9.0,2.0,NaN,1,"Trastornos neuróticos, trastornos relacionados...",756,19,1,2,M,0.609264,2771,10,2.35,media,5,3,0,1,0,0,0,0
2,-5406560181117020160,8998349.0,andalucia,17490445801063320188,2016-01-01,2016-01-01,2016-01-11,NaN,1,1,domicilio,10,1,PSQ,1.0,21.0,9.0,2.0,NaN,1,Trastornos de la personalidad y del comportami...,752,19,2,1,M,0.881297,4009,10,0.55,baja,1,0,0,0,1,0,0,0
3,-1823171082,8800205.0,andalucia,-3960068041784730112,2016-01-01,2016-01-01,2016-01-27,NaN,1,1,domicilio,26,1,PSQ,1.0,21.0,9.0,2.0,NaN,1,"Esquizofrenia, trastornos esquizotípicos y tra...",750,19,1,2,M,1.335036,6073,10,1.75,media,5,0,1,0,0,0,0,0
4,-2828047377,8745063.0,andalucia,-3960068041784730112,2016-01-01,2016-01-01,2016-01-18,NaN,1,1,domicilio,17,1,PSQ,1.0,21.0,9.0,2.0,NaN,1,Trastornos de la personalidad y del comportami...,752,19,1,1,M,0.850111,3867,10,0.30,baja,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21205,11802005621319091719,2534956.0,andalucia,-2088791444897189888,2018-12-31,2018-12-31,2019-01-01,NaN,1,1,domicilio,1,0,PSQ,1.0,21.0,9.0,2.0,NaN,12,"Esquizofrenia, trastornos esquizotípicos y tra...",750,19,2,1,M,1.609419,7613,10,1.75,media,5,0,1,0,0,0,0,0
21206,-423452310915088000,2585147.0,andalucia,-924212801566861056,2018-12-31,2018-12-31,2019-02-06,NaN,1,1,domicilio,37,1,PSQ,1.0,21.0,9.0,2.0,NaN,12,Trastornos mentales y del comportamiento debid...,775,20,2,1,M,0.888013,4200,10,1.75,media,5,0,0,0,0,1,0,0
21207,1160647917-1937584955,2643973.0,andalucia,-2088791444897189888,2018-12-31,2018-12-31,2019-01-10,NaN,1,1,domicilio,10,1,PSQ,1.0,21.0,9.0,2.0,NaN,12,Trastornos de la personalidad y del comportami...,752,19,2,1,M,0.925525,4378,10,4.15,muy alta,13,0,0,0,1,0,0,0
21208,NaN,2573014.0,andalucia,-2940034890,2018-12-31,2018-12-31,2019-01-04,NaN,1,3,alta voluntaria,4,0,PSQ,9.0,21.0,9.0,2.0,NaN,12,"Esquizofrenia, trastornos esquizotípicos y tra...",750,19,2,1,M,1.609419,7613,10,1.45,media,4,0,1,0,0,0,0,0


In [49]:
# Verificar integridad referencial entre las tablas
print("VERIFICACIÓN DE INTEGRIDAD REFERENCIAL")
print("="*50)

# Verificar que todos los cip_sns_recodificado en ingresos existen en pacientes
cips_ingresos = set(df_ingresos['cip_sns_recodificado'])
cips_pacientes = set(df_pacientes['cip_sns_recodificado'])

cips_huerfanos = cips_ingresos - cips_pacientes
if len(cips_huerfanos) == 0:
    print("✓ Integridad referencial correcta: Todos los CIP de ingresos existen en pacientes")
else:
    print(f"⚠️ {len(cips_huerfanos)} CIP en ingresos no encontrados en pacientes")

# Verificar coherencia de datos entre tablas para algunos pacientes
print(f"\nVERIFICACIÓN DE COHERENCIA DE DATOS")
print("-" * 40)

# Tomar una muestra de pacientes con múltiples ingresos
pacientes_multiples_ingresos = ingresos_por_paciente[ingresos_por_paciente > 1].head(3).index

for cip in pacientes_multiples_ingresos:
    paciente_info = df_pacientes[df_pacientes['cip_sns_recodificado'] == cip]
    ingresos_info = df_ingresos[df_ingresos['cip_sns_recodificado'] == cip]
    
    print(f"\nPaciente {cip}:")
    print(f"  - Sexo en tabla pacientes: {paciente_info['sexo'].iloc[0]}")
    print(f"  - Edad en tabla pacientes: {paciente_info['edad'].iloc[0]}")
    print(f"  - Número de ingresos: {len(ingresos_info)}")
    print(f"  - Fechas de ingreso: {sorted(ingresos_info['fecha_ingreso'].tolist())}")

print(f"\nRESUMEN DE NORMALIZACIÓN COMPLETADA")
print("="*50)
print(f"📊 TABLAS CREADAS:")
print(f"  1. df_pacientes: {len(df_pacientes)} pacientes únicos")
print(f"  2. df_ingresos: {len(df_ingresos)} ingresos/episodios")
print(f"  3. df_procedimientos: {len(df_procedimientos)} tipos de procedimientos")
print(f"  4. df_diagnosticos: {len(df_diagnosticos)} tipos de diagnósticos")
print(f"  5. df_procedimientos_pacientes: {len(df_procedimientos_pacientes)} relaciones procedimiento-paciente")
print(f"  6. df_diagnosticos_pacientes: {len(df_diagnosticos_pacientes)} relaciones diagnóstico-paciente")

print(f"\n🔗 ESTRUCTURA RELACIONAL:")
print(f"  - Pacientes (1) → Ingresos (N) via cip_sns_recodificado")
print(f"  - Pacientes (N) → Procedimientos (M) via df_procedimientos_pacientes")
print(f"  - Pacientes (N) → Diagnósticos (M) via df_diagnosticos_pacientes")
print(f"  - Procedimientos → Descripción via df_procedimientos")
print(f"  - Diagnósticos → Descripción via df_diagnosticos")

VERIFICACIÓN DE INTEGRIDAD REFERENCIAL
✓ Integridad referencial correcta: Todos los CIP de ingresos existen en pacientes

VERIFICACIÓN DE COHERENCIA DE DATOS
----------------------------------------

Paciente 16029552631703118806:
  - Sexo en tabla pacientes: varon
  - Edad en tabla pacientes: 39
  - Número de ingresos: 29
  - Fechas de ingreso: ['2016-04-17', '2016-05-08', '2016-05-30', '2016-07-17', '2016-09-22', '2016-11-09', '2016-12-13', '2017-01-29', '2017-03-11', '2017-03-22', '2017-04-09', '2017-11-13', '2017-11-28', '2017-12-31', '2018-01-09', '2018-01-18', '2018-01-31', '2018-03-02', '2018-03-09', '2018-03-21', '2018-06-15', '2018-07-20', '2018-08-19', '2018-09-07', '2018-10-06', '2018-11-07', '2018-11-27', '2018-12-05', '2018-12-22']

Paciente 1479231345-1231192323:
  - Sexo en tabla pacientes: mujer
  - Edad en tabla pacientes: 49
  - Número de ingresos: 28
  - Fechas de ingreso: ['2017-10-12', '2017-10-18', '2018-01-04', '2018-02-06', '2018-02-13', '2018-02-20', '2018-02-2

In [50]:
# VERIFICACIÓN DE COBERTURA DE TODAS LAS COLUMNAS ORIGINALES
print("VERIFICACIÓN DE COBERTURA DE COLUMNAS")
print("="*50)

# Lista de todas las columnas originales proporcionadas por el usuario
columnas_originales = [
    'comunidad_autonoma', 'fecha_nacimiento', 'sexo', 'fecha_ingreso', 'circunstancia_contacto', 
    'fecha_fin_contacto', 'tipo_alta', 'estancia_dias', 'diagnostico_principal', 'categoria', 
    'diagnostico_2', 'diagnostico_3', 'diagnostico_4', 'diagnostico_5', 'diagnostico_6', 
    'diagnostico_7', 'diagnostico_8', 'diagnostico_9', 'diagnostico_10', 'diagnostico_11', 
    'diagnostico_12', 'diagnostico_13', 'diagnostico_14', 'fecha_intervencion', 'procedimiento_1', 
    'procedimiento_2', 'procedimiento_3', 'procedimiento_4', 'procedimiento_5', 'procedimiento_6', 
    'procedimiento_7', 'procedimiento_8', 'procedimiento_9', 'procedimiento_10', 'procedimiento_11', 
    'procedimiento_12', 'procedimiento_13', 'procedimiento_14', 'procedimiento_15', 'procedimiento_16', 
    'procedimiento_17', 'procedimiento_18', 'procedimiento_19', 'procedimiento_20', 'grd_apr', 
    'cdm_apr', 'nivel_severidad_apr', 'riesgo_mortalidad_apr', 'servicio', 'edad', 'coste_apr', 
    'cie', 'numero_registro_anual', 'centro_recodificado', 'cip_sns_recodificado', 'pais_nacimiento', 
    'pais_residencia', 'fecha_inicio_contacto', 'regimen_financiacion', 'procedencia', 
    'continuidad_asistencial', 'ingreso_en_uci', 'dias_uci', 'diagnostico_15', 'diagnostico_16', 
    'diagnostico_17', 'diagnostico_18', 'diagnostico_19', 'diagnostico_20', 'poa_diagnostico_principal', 
    'poa_diagnostico_2', 'poa_diagnostico_3', 'poa_diagnostico_4', 'poa_diagnostico_5', 
    'poa_diagnostico_6', 'poa_diagnostico_7', 'poa_diagnostico_8', 'poa_diagnostico_9', 
    'poa_diagnostico_10', 'poa_diagnostico_11', 'poa_diagnostico_12', 'poa_diagnostico_13', 
    'poa_diagnostico_14', 'poa_diagnostico_15', 'poa_diagnostico_16', 'poa_diagnostico_17', 
    'poa_diagnostico_18', 'poa_diagnostico_19', 'poa_diagnostico_20', 'procedimiento_externo_1', 
    'procedimiento_externo_2', 'procedimiento_externo_3', 'tipo_grd_apr', 'peso_espanol_apr', 
    'edad_en_ingreso', 'mes_ingreso', 'tipo_alta_desc', 'menor_edad', 'nacido_espana', 
    'larga_estancia', 'cat_esquizofrenia', 'cat_trastorno_neurotico', 'cat_trastorno_personalidad', 
    'cat_trastorno_mental', 'cat_trastorno_humor', 'cat_trastorno_emocional', 
    'num_diagnosticos_secundarios', 'num_procedimientos', 'complejidad_general', 
    'categoria_complejidad', 'grupo_etario'
]

print(f"Total de columnas originales a verificar: {len(columnas_originales)}")

# Obtener columnas de cada tabla normalizada
columnas_df_pacientes = list(df_pacientes.columns)
columnas_df_ingresos = list(df_ingresos.columns)
columnas_df_diagnosticos = ['ID', 'descripcion']  # Del catálogo
columnas_df_diagnosticos_pacientes = list(df_diagnosticos_pacientes.columns)
columnas_df_procedimientos = ['ID', 'descripcion']  # Del catálogo
columnas_df_procedimientos_pacientes = list(df_procedimientos_pacientes.columns)

print(f"\nColumnas en df_pacientes ({len(columnas_df_pacientes)}):")
print(f"  {columnas_df_pacientes}")

print(f"\nColumnas en df_ingresos ({len(columnas_df_ingresos)}):")
print(f"  {columnas_df_ingresos}")

print(f"\nColumnas en df_diagnosticos_pacientes ({len(columnas_df_diagnosticos_pacientes)}):")
print(f"  {columnas_df_diagnosticos_pacientes}")

print(f"\nColumnas en df_procedimientos_pacientes ({len(columnas_df_procedimientos_pacientes)}):")
print(f"  {columnas_df_procedimientos_pacientes}")

# Crear conjunto de todas las columnas guardadas en las tablas normalizadas
# Excluir cip_sns_recodificado duplicado y las columnas de catálogos (ID, descripcion)
todas_columnas_guardadas = set()

# De pacientes (excluir cip_sns_recodificado que se repite)
todas_columnas_guardadas.update([col for col in columnas_df_pacientes])

# De ingresos (excluir cip_sns_recodificado que es FK)
todas_columnas_guardadas.update([col for col in columnas_df_ingresos if col != 'cip_sns_recodificado'])

# Las columnas de diagnósticos y procedimientos se transformaron en relaciones
# Verificar que las originales están representadas
diagnosticos_originales = [col for col in columnas_originales if col.startswith('diagnostico_')]
poa_diagnosticos_originales = [col for col in columnas_originales if col.startswith('poa_diagnostico_')]
procedimientos_originales = [col for col in columnas_originales if col.startswith('procedimiento_')]

print(f"\n--- ANÁLISIS DE TRANSFORMACIONES ---")
print(f"Diagnósticos originales: {len(diagnosticos_originales)} → Transformados a df_diagnosticos_pacientes")
print(f"POA diagnósticos originales: {len(poa_diagnosticos_originales)} → Integrados en df_diagnosticos_pacientes") 
print(f"Procedimientos originales: {len(procedimientos_originales)} → Transformados a df_procedimientos_pacientes")

# Considerar las columnas transformadas como "guardadas"
todas_columnas_guardadas.update(diagnosticos_originales)
todas_columnas_guardadas.update(poa_diagnosticos_originales)
todas_columnas_guardadas.update(procedimientos_originales)

print(f"\n--- VERIFICACIÓN FINAL ---")

# Encontrar columnas originales que NO están en ninguna tabla
columnas_no_guardadas = set(columnas_originales) - todas_columnas_guardadas

if columnas_no_guardadas:
    print(f"❌ COLUMNAS NO GUARDADAS ({len(columnas_no_guardadas)}):")
    for col in sorted(columnas_no_guardadas):
        print(f"  - {col}")
else:
    print("✅ TODAS LAS COLUMNAS ORIGINALES ESTÁN GUARDADAS")

print(f"\nRESUMEN:")
print(f"  - Columnas originales: {len(columnas_originales)}")
print(f"  - Columnas guardadas: {len(todas_columnas_guardadas)}")
print(f"  - Columnas faltantes: {len(columnas_no_guardadas)}")

# Mostrar distribución por tabla
print(f"\n--- DISTRIBUCIÓN POR TABLA ---")
print(f"  - df_pacientes: {len(columnas_df_pacientes)} columnas")
print(f"  - df_ingresos: {len([col for col in columnas_df_ingresos if col != 'cip_sns_recodificado'])} columnas (+ 1 FK)")
print(f"  - Diagnósticos transformados: {len(diagnosticos_originales + poa_diagnosticos_originales)} columnas")
print(f"  - Procedimientos transformados: {len(procedimientos_originales)} columnas")

VERIFICACIÓN DE COBERTURA DE COLUMNAS
Total de columnas originales a verificar: 111

Columnas en df_pacientes (10):
  ['cip_sns_recodificado', 'fecha_nacimiento', 'sexo', 'pais_nacimiento', 'pais_residencia', 'edad', 'edad_en_ingreso', 'menor_edad', 'nacido_espana', 'grupo_etario']

Columnas en df_ingresos (39):
  ['cip_sns_recodificado', 'numero_registro_anual', 'comunidad_autonoma', 'centro_recodificado', 'fecha_ingreso', 'fecha_inicio_contacto', 'fecha_fin_contacto', 'fecha_intervencion', 'circunstancia_contacto', 'tipo_alta', 'tipo_alta_desc', 'estancia_dias', 'larga_estancia', 'servicio', 'regimen_financiacion', 'procedencia', 'continuidad_asistencial', 'ingreso_en_uci', 'dias_uci', 'mes_ingreso', 'categoria', 'grd_apr', 'cdm_apr', 'nivel_severidad_apr', 'riesgo_mortalidad_apr', 'tipo_grd_apr', 'peso_espanol_apr', 'coste_apr', 'cie', 'complejidad_general', 'categoria_complejidad', 'num_diagnosticos_secundarios', 'num_procedimientos', 'cat_esquizofrenia', 'cat_trastorno_neurotico',

In [ ]:
# GUARDAR TABLAS NORMALIZADAS EN FORMATO CSV
print("GUARDANDO TABLAS NORMALIZADAS")
print("="*40)

import os

# Crear directorio si no existe
output_dir = '../definitive_data'
os.makedirs(output_dir, exist_ok=True)

# Lista de DataFrames a guardar
dataframes_to_save = [
    ('df_diagnosticos', df_diagnosticos),
    ('df_diagnosticos_pacientes', df_diagnosticos_pacientes),
    ('df_pacientes', df_pacientes),
    ('df_ingresos', df_ingresos),
    ('df_procedimientos', df_procedimientos),
    ('df_procedimientos_pacientes', df_procedimientos_pacientes)
]

# Guardar cada DataFrame
for df_name, df_data in dataframes_to_save:
    file_path = os.path.join(output_dir, f'{df_name}.csv')
    df_data.to_csv(file_path, index=False, encoding='utf-8')
    print(f"✓ {df_name}.csv guardado - {len(df_data)} filas, {len(df_data.columns)} columnas")

print(f"\n📁 Todos los archivos guardados en: {os.path.abspath(output_dir)}")

# Verificar tamaños de archivos
print(f"\n📊 RESUMEN DE ARCHIVOS:")
for df_name, df_data in dataframes_to_save:
    file_path = os.path.join(output_dir, f'{df_name}.csv')
    file_size = os.path.getsize(file_path) / 1024  # KB
    print(f"  - {df_name}.csv: {file_size:.1f} KB")

print(f"\n✅ EXPORTACIÓN COMPLETADA")